In [ ]:
import tkinter as tk
from tkinter import messagebox, filedialog
import pandas as pd
from sklearn.linear_model import LinearRegression
import random
import numpy as np

# Define the correct admin password
admin_password = "0000"

# Define the model and column names as global variables
model = None
model_columns = None


def predict_grades_single(features, previous_marks=None):
    global model, model_columns  # Access the global model and column names

    # Load the dataset into a Pandas DataFrame
    data = pd.read_csv('exams.csv')  # Replace 'your_dataset.csv' with the actual filename/path

    # Remove trailing whitespaces from column names
    data.columns = data.columns.str.strip()

    # Select the relevant columns for training and prediction
    selected_features = ['Student Number', 'Attendance (%)', 'Ethnicity', 'Gender', 'Parental Level of Education', 'Lunch Type', 'Test Preparation Course']
    target_variables = ['Mathematics', 'English', 'Biology', 'Physics', 'Chemistry', 'ICT']

    # Convert categorical variables into numerical representations (one-hot encoding)
    data_encoded = pd.get_dummies(data[selected_features])

    # Create a linear regression model and fit it to the entire dataset
    model = LinearRegression()
    model.fit(data_encoded, data[target_variables])

    # Store the column names
    model_columns = data_encoded.columns

    # Prepare input data for prediction
    input_data = pd.DataFrame([features], columns=selected_features)
    input_data_encoded = pd.get_dummies(input_data)

    # Check for missing columns in input_data_encoded
    missing_cols = set(model_columns) - set(input_data_encoded.columns)
    for col in missing_cols:
        input_data_encoded[col] = 0

    # Reorder columns to match the order in the trained model
    input_data_encoded = input_data_encoded[model_columns]

    # Make predictions
    predictions = model.predict(input_data_encoded)

    return dict(zip(target_variables, predictions[0]))


def randomize_grades(data, target_variables):
    randomized_grades_list = []
    for _, row in data.iterrows():
        randomized_grades = {}
        for subject in target_variables:
            grade = row[subject]
            lower_limit = max(0, int(round(grade)) - 4)
            upper_limit = min(100, int(round(grade)) + 4)
            randomized_grades[subject] = random.randint(lower_limit, upper_limit)
        randomized_grades_list.append(randomized_grades)

    return pd.DataFrame(randomized_grades_list, columns=target_variables)

def predict_grades_bulk(file_path):
    # Load the dataset into a Pandas DataFrame
    data = pd.read_csv(file_path)

    # Remove trailing whitespaces from column names
    data.columns = data.columns.str.strip()

    # Define the target variables
    target_variables = ['Mathematics', 'English', 'Biology', 'Physics', 'Chemistry', 'ICT']

    # Randomize the grades for the entire dataset
    randomized_data = randomize_grades(data, target_variables)

    # Create a new DataFrame with randomized grades and student numbers
    output_data = pd.DataFrame(randomized_data, columns=target_variables)
    output_data['Student Number'] = data['Student Number']

    # Calculate additional statistics subject-wise
    avg_marks = output_data.mean()
    highest_marks = output_data.max()
    lowest_marks = output_data.min()
    num_failed = (output_data < 50).sum()
    num_passed = (output_data >= 50).sum()

    # Add statistics as new rows at the bottom of the data
    statistics_names = ['Average Marks', 'Highest Marks', 'Lowest Marks', 'Number of Failed', 'Number of Passed']
    statistics_data = pd.DataFrame([avg_marks, highest_marks, lowest_marks, num_failed, num_passed], columns=target_variables)
    statistics_data.insert(0, 'Subject', statistics_names)

    output_data = pd.concat([output_data, statistics_data], ignore_index=True)

    return output_data

def login_button_click():
    entered_username = entry_username.get()
    entered_password = entry_password.get()

    if entered_password == admin_password:
        # Clear the password entry field
        entry_password.delete(0, tk.END)

        # Hide the login page
        frame_login.pack_forget()

        # Show the prediction page
        frame_prediction.pack()
    else:
        messagebox.showerror("Error", "Incorrect password. Access denied.")

def randomize_grades_single(features):
    randomized_grades = {}
    for subject, grade in features.items():
        if isinstance(grade, str):
            randomized_grades[subject] = "Invalid input: Please enter numeric grades."
        else:
            grade = float(grade)  # Convert grade to float
            lower_limit = max(0, int(round(grade)) - 5)
            upper_limit = min(100, int(round(grade)) + 5)
            randomized_grades[subject] = random.randint(lower_limit, upper_limit)

    return randomized_grades

def predict_button_click():
    features = {
        'Student Number': int(entry_student_number.get()),
        'Attendance (%)': float(entry_attendance.get()),
        'Gender': entry_gender.get(),
        'Ethnicity': entry_race.get(),
        'Parental Level of Education': entry_education.get(),
        'Lunch Type': entry_lunch.get(),
        'Test Preparation Course': entry_prep.get(),
        'Mathematics': entry_math.get(),
        'English': entry_eng.get(),
        'Biology': entry_bio.get(),
        'Physics': entry_phy.get(),
        'Chemistry': entry_chem.get(),
        'ICT': entry_ict.get()
    }

    # Randomize grades within the range of +5 to -5 of the input grades
    randomized_grades = randomize_grades_single(features)

    # Display the randomized grades
    formatted_grades = "\n".join([f"{subject}: {grade}" for subject, grade in randomized_grades.items()])
    result_text.set("Randomized Grades:\n" + formatted_grades)

    
def bulk_predict_button_click():
    file_path = filedialog.askopenfilename(filetypes=[("CSV files", "*.csv")])
    if file_path:
        try:
            output_data = predict_grades_bulk(file_path)
            output_file_path = filedialog.asksaveasfilename(defaultextension=".csv", filetypes=[("CSV files", "*.csv")])
            if output_file_path:
                output_data.to_csv(output_file_path, index=False)
                messagebox.showinfo("Bulk Prediction", "Prediction completed. Output CSV file saved successfully.")
        except Exception as e:
            messagebox.showerror("Error", str(e))
    else:
        messagebox.showwarning("Bulk Prediction", "No file selected.")


def clear_button_click():
    global single_prediction_result  # Access the global single_prediction_result variable
    # Clear all input fields
    entry_student_number.delete(0, tk.END)
    entry_attendance.delete(0, tk.END)
    entry_gender.delete(0, tk.END)
    entry_race.delete(0, tk.END)
    entry_education.delete(0, tk.END)
    entry_lunch.delete(0, tk.END)
    entry_prep.delete(0, tk.END)
    entry_math.delete(0, tk.END)
    entry_eng.delete(0, tk.END)
    entry_bio.delete(0, tk.END)
    entry_phy.delete(0, tk.END)
    entry_chem.delete(0, tk.END)
    entry_ict.delete(0, tk.END)
    result_text.set("")  # Clear the result_text variable
    single_prediction_result = None  # Reset the single_prediction_result variable

def logout_button_click():
    # Hide the prediction page
    frame_prediction.pack_forget()

    # Show the login page
    frame_login.pack()

# Create the UI window
window = tk.Tk()
window.title("Grade Prediction")
window.geometry("1000x1000")

# Create the login page
frame_login = tk.Frame(window, bg="#506180")

label_username = tk.Label(frame_login, text="Enter Username:", font=("Arial", 14), bg="#506180")
label_username.pack()
entry_username = tk.Entry(frame_login, font=("Arial", 14))
entry_username.pack()

label_password = tk.Label(frame_login, text="Enter Password:", font=("Arial", 14), bg="#506180")
label_password.pack()
entry_password = tk.Entry(frame_login, show="*", font=("Arial", 14))
entry_password.pack()

button_login = tk.Button(frame_login, text="Login", command=login_button_click, font=("Arial", 14))
button_login.pack()

# Create the prediction page (initially hidden)
frame_prediction = tk.Frame(window, bg="#506180")

label_student_number = tk.Label(frame_prediction, text="Student Number", font=("Arial", 14), bg="#506180")
label_student_number.pack()
entry_student_number = tk.Entry(frame_prediction, font=("Arial", 14))
entry_student_number.pack(pady=10)

label_attendance = tk.Label(frame_prediction, text="Attendance (%)", font=("Arial", 14), bg="#506180")
label_attendance.pack()
entry_attendance = tk.Entry(frame_prediction, font=("Arial", 14))
entry_attendance.pack(pady=10)

label_gender = tk.Label(frame_prediction, text="Gender", font=("Arial", 14), bg="#506180")
label_gender.pack()
entry_gender = tk.Entry(frame_prediction, font=("Arial", 14))
entry_gender.pack(pady=10)

label_race = tk.Label(frame_prediction, text="Ethnicity", font=("Arial", 14), bg="#506180")
label_race.pack()
entry_race = tk.Entry(frame_prediction, font=("Arial", 14))
entry_race.pack(pady=10)

label_education = tk.Label(frame_prediction, text="Parental Level of Education", font=("Arial", 14), bg="#506180")
label_education.pack()
entry_education = tk.Entry(frame_prediction, font=("Arial", 14))
entry_education.pack(pady=10)

label_lunch = tk.Label(frame_prediction, text="Lunch Type", font=("Arial", 14), bg="#506180")
label_lunch.pack()
entry_lunch = tk.Entry(frame_prediction, font=("Arial", 14))
entry_lunch.pack(pady=10)

label_prep = tk.Label(frame_prediction, text="Test Preparation Course", font=("Arial", 14), bg="#506180")
label_prep.pack()
entry_prep = tk.Entry(frame_prediction, font=("Arial", 14))
entry_prep.pack(pady=10)

label_previous_grades = tk.Label(frame_prediction, text="Previous Grades", font=("Arial", 14), bg="#506180")
label_previous_grades.pack(pady=5)

input_frame = tk.Frame(frame_prediction, bg="#506180")
input_frame.pack()

label_math = tk.Label(input_frame, text="Mathematics", font=("Arial", 12), bg="#506180")
label_math.grid(row=0, column=0, padx=5, pady=5)
entry_math = tk.Entry(input_frame, font=("Arial", 12), width=8)
entry_math.grid(row=0, column=1, padx=5, pady=5)

label_eng = tk.Label(input_frame, text="English", font=("Arial", 12), bg="#506180")
label_eng.grid(row=0, column=2, padx=5, pady=5)
entry_eng = tk.Entry(input_frame, font=("Arial", 12), width=8)
entry_eng.grid(row=0, column=3, padx=5, pady=5)

label_bio = tk.Label(input_frame, text="Biology", font=("Arial", 12), bg="#506180")
label_bio.grid(row=0, column=4, padx=5, pady=5)
entry_bio = tk.Entry(input_frame, font=("Arial", 12), width=8)
entry_bio.grid(row=0, column=5, padx=5, pady=5)

label_phy = tk.Label(input_frame, text="Physics", font=("Arial", 12), bg="#506180")
label_phy.grid(row=0, column=6, padx=5, pady=5)
entry_phy = tk.Entry(input_frame, font=("Arial", 12), width=8)
entry_phy.grid(row=0, column=7, padx=5, pady=5)

label_chem = tk.Label(input_frame, text="Chemistry", font=("Arial", 12), bg="#506180")
label_chem.grid(row=0, column=8, padx=5, pady=5)
entry_chem = tk.Entry(input_frame, font=("Arial", 12), width=8)
entry_chem.grid(row=0, column=9, padx=5, pady=5)

label_ict = tk.Label(input_frame, text="ICT", font=("Arial", 12), bg="#506180")
label_ict.grid(row=0, column=10, padx=5, pady=5)
entry_ict = tk.Entry(input_frame, font=("Arial", 12), width=8)
entry_ict.grid(row=0, column=11, padx=5, pady=5)

button_frame = tk.Frame(frame_prediction, bg="#506180")
button_frame.pack()

button_predict = tk.Button(button_frame, text="Single Prediction", command=predict_button_click, font=("Calibri", 14))
button_predict.pack(side=tk.LEFT, padx=10, pady=10)

button_bulk_predict = tk.Button(button_frame, text="Bulk Prediction", command=bulk_predict_button_click, font=("Calibri", 14))
button_bulk_predict.pack(side=tk.LEFT, padx=10, pady=10)

button_clear = tk.Button(button_frame, text="Clear", command=clear_button_click, font=("Calibri", 14))
button_clear.pack(side=tk.LEFT, padx=10, pady=10)

button_logout = tk.Button(button_frame, text="Log Out", command=logout_button_click, font=("Calibri", 14))
button_logout.pack(side=tk.LEFT, padx=10, pady=10)

result_text = tk.StringVar()
result_label = tk.Label(frame_prediction, textvariable=result_text, font=("Arial", 14), justify='left', bg="#506180")
result_label.pack(pady=10)

# Run the UI
frame_login.pack(fill="both", expand=True)
window.mainloop()


/Users/hamzashaheen/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
